# 3b - Training with built-in XGBoost <a class="anchor" id="top"></a>
* [Introduction](#intro)
* [Setup](#setup)
* [Estimator creation](#estim)
    * [Define estimator](#define)
    * [Train estimator and tune parameters](#tune)
* [Evaluate training result](#eval)
* [Cleanup resources](#clean)

## Introduction <a class="anchor" id="intro"></a>
In this notebook, we will train an XGBoost model, evaluate the training performance, and output model artifacts.
XGBoost is a regularizing gradient boosting framework with proven performance in ML competitions.
Additionally, the algorithm has a variety of hyperparameters, which allows for signigicant gains to be had by way of tuning.
We will be using Amazon's built-in XGBoost implementation, which integrates nicely with their hyperparameter tuner.

## Setup <a class="anchor" id="setup"></a>
First, we import Sageamker SDK dependencies as well as modules used in application below.
We also get relevant sessions and read in local environment data.

In [1]:
import json
import uuid
import boto3
import random
import tarfile
import pickle as pkl
import datetime as dt
import sagemaker as sm
import sagemaker.xgboost as xgb

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
sns.set_style("darkgrid")

In [2]:
sm_session = sm.Session()
role = sm.get_execution_role()
boto3_session = boto3.session.Session()
now = dt.datetime.now().strftime(r"%Y%m%dT%H%M%S")

In [3]:
# Get boto3 session attributes.
account = boto3_session.client("sts").get_caller_identity()["Account"]
region = boto3_session.region_name

# Create clients to access S3.
s3_client = boto3_session.client("s3")
s3_resource = boto3_session.resource("s3")

In [4]:
# Retrieve data and model bucket names.
with open("/home/ec2-user/.aiml-bb/stack-data.json", "r") as f:
    data = json.load(f)
    data_bucket = data["data_bucket"]
    model_bucket = data["model_bucket"]

## Estimator creation <a class="anchor" id="estim"></a>
We can now create the XGBoost estimator, using Amazon's built-in implementation.
Because the model is managed for us, there is little to do in way of setup.

### Define estimator <a class="anchor" id="define"></a>
Here we create the `Estimator` object, and all resources that are required to do so.

In [5]:
# Get XGBoost container image for current region.
xgb_container_image = sm.image_uris.retrieve("xgboost", region, "latest")

# Create a unique training job name.
training_job_name = f"xgboost-{str(uuid.uuid4())[:8]}"

In [6]:
train_input = sm.inputs.TrainingInput(
    s3_data=f"s3://{data_bucket}/preprocessing_output/train/", 
    content_type="csv"
)
validation_input = sm.inputs.TrainingInput(
    s3_data=f"s3://{data_bucket}/preprocessing_output/validation/",
    content_type="csv"
)

In [7]:
# Create estimator running the XGBoost container.
xgb_estimator = sm.estimator.Estimator(
    xgb_container_image,
    role, 
    instance_count=1, 
    instance_type="ml.m5.12xlarge",
    volume_size=50,
    output_path=f"s3://{model_bucket}/sagemaker-xgboost/"
)

### Train estimator and tune parameters <a class="anchor" id="tune"></a>
Hyperparameters are now defined as well as valid ranges.
We then attach a `HyperparameterTuner` and fit the tuner.

In [8]:
# Define starting hyperparameters for the model.
xgb_estimator.set_hyperparameters(
    eval_metric="auc",
    objective="binary:logistic",
    max_depth=6,
    eta=0.2,
    alpha=2.0,
    gamma=4,
    min_child_weight=8,
    subsample=0.8,
    silent=0,
    num_round=100,
    early_stopping_rounds=25
)
# Set ranges of XGBoost hyperparameters for tuning.
xgb_hyperparameter_ranges = {
    "eta": sm.tuner.ContinuousParameter(0, 1),
    "alpha": sm.tuner.ContinuousParameter(0, 3),
    "min_child_weight": sm.tuner.ContinuousParameter(1, 10),
    "max_depth": sm.tuner.IntegerParameter(1, 10)
}

In [ ]:
# Create tuner and fit.
xgb_objective_metric_name = "validation:auc"
xgb_tuner = sm.tuner.HyperparameterTuner(
    xgb_estimator,
    xgb_objective_metric_name,
    xgb_hyperparameter_ranges,
    max_jobs=10,
    max_parallel_jobs=5,
    strategy="Bayesian"
)
xgb_tuner.fit(
    {"train": train_input, "validation": validation_input}
)

............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Select best model <a class="anchor" id="select-best"></a>
View the analytics from the tuning job and select the best model.
We then store the best model in location that we can reference more easily in other notebooks.

In [ ]:
# View analytics on tuning job results.
xgb_tuner_analytics = sm.HyperparameterTuningJobAnalytics(
    xgb_tuner.describe()["HyperParameterTuningJobName"]
)
xgb_tuner_analytics.dataframe().sort_values("FinalObjectiveValue")

In [ ]:
# Take best model and copy to location we can reference in other notebooks.
xgb_best_training_job_name = xgb_tuner.best_training_job()
xgb_best_training_job_key = f"sagemaker-xgboost/{xgb_best_training_job_name}/output/model.tar.gz"
copy_source = {
    "Bucket": model_bucket,
    "Key": xgb_best_training_job_key
}
s3_resource.Bucket(model_bucket).copy(
    copy_source, 
    "sagemaker-xgboost-tuned/model.tar.gz"
)

## Create endpoint to test model <a class="anchor" id="endpoint"></a>
To test the model, we must now create an endpoint that we can send the test data set aside during preprocessing.

In [ ]:
# Create model and endpoint from best fitted estimator above.
xgb_model = sm.model.Model(
    image_uri=xgb_container_image,
    model_data=f"s3://{model_bucket}/sagemaker-xgboost-tuned/model.tar.gz",
    role=role
)
endpoint_name = f"xgb-test-endpt-{now}"
xgb_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name=endpoint_name
)

In [ ]:
# Connect a predictor to the endpoint for inference.
xgb_predictor = sm.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sm_session,
    serializer=sm.serializers.CSVSerializer(
        content_type="text/csv"
    )
)

In [ ]:
# Iterate over testing data and compute statistics.
list_objs_response = s3_client.list_objects_v2(
    Bucket=data_bucket, 
    Prefix="preprocessing_output/train"
)

# Arrays to keep track of results.
test_actuals = []
test_predictions = []
for obj in list_objs_response["Contents"]:
    
    # Iterate over lines in object contents via stream.
    obj_resource = s3_resource.Object(data_bucket, obj["Key"])
    for line in obj_resource.get()["Body"].iter_lines():
        target, features = line.decode("utf-8").split(",", maxsplit=1)
        features = features.strip()
        prediction = xgb_predictor.predict(features)
        
        test_actuals.append(float(target))
        test_predictions.append(float(prediction))
    
        if len(test_actuals) > 100_000:
            break

## Evaluate training results <a class="anchor" id="eval"></a>
Lastly, we evaluate the results of training against the testing data set.
Note that this data set is not included in training and has never been seen by the model.

In [ ]:
test_actuals = [0,0,0,1,0,0,1,1,1]
test_predictions = [0,0,1,1,0,0,1,0,1]

In [ ]:
# Wrap lists in numpy arrays for analysis.
test_actuals_np = np.array(test_actuals)
test_predictions_np = np.array(test_predictions)

In [ ]:
# Compute summary statistics on perfomance.
performance_statistics = {
    "accuracy": metrics.accuracy_score(test_actuals_np, test_predictions_np),
    "precision": metrics.precision_score(test_actuals_np, test_predictions_np),
    "recall": metrics.recall_score(test_actuals_np, test_predictions_np),
    "f1": metrics.f1_score(test_actuals_np, test_predictions_np),
    "auc": metrics.roc_auc_score(test_actuals_np, test_predictions_np),
}
print(json.dumps(performance_statistics, indent=4))

In [ ]:
# Compute confusion matrix.
confusion_df = pd.crosstab(
    test_actuals_np, 
    test_predictions_np, 
    rownames=["Actuals"], 
    colnames=["Predictions"]
)
norm_confusion_df = confusion_df / confusion_df.sum(axis=1)

# Show confusion matrix.
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    norm_confusion_df, 
    vmin=-1.0, vmax=1.0, annot=True, fmt=".2f", 
    ax=ax
)
ax.set_title("Confusion matrix of testing results")
plt.show()

In [ ]:
# Compute ROC curve.
fpr, tpr, thresholds = metrics.roc_curve(test_actuals_np, test_predictions_np)

# Plot ROC matrix.
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fpr, tpr, label='ROC')
ax.plot([0, 1], [0, 1], linestyle='--')
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.set_title('Receiver operating characteristic curve')
ax.legend()
plt.show()

## Cleanup resources <a class="anchor" id="clean"></a>
Because this is a temporary project, delete the endpoint.

In [ ]:
sm_session.delete_endpoint(endpoint_name)